In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

0it [00:00, ?it/s]Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to ./data/MNIST/raw/train-images-idx3-ubyte.gz
 99%|█████████▉| 9805824/9912422 [00:03<00:00, 8704886.12it/s]Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

0it [00:00, ?it/s]Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz to ./data/MNIST/raw/train-labels-idx1-ubyte.gz

 85%|████████▌ | 24576/28881 [00:01<00:00, 214905.96it/s]

0it [00:00, ?it/s]Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


  0%|          | 0/1648877 [00:00<?, ?it/s]

  1%|          | 16384/1648877 [00:00<00:25, 63672.18it/s]

  3%|▎         | 49152/1648877 [00:00<00:10, 149191.73it/s]

  6%|▌         | 98304/1648877 [00:01<00:07, 213367.04it/s]

 12%|█▏        | 204800/1648877 [00:01<00:03, 433093.58it/s]

 15%|█▌        | 253952/1648877 [00:01<00:03, 435546.97it/s]

 24%|██▍       | 401408/1648877 [00:01<00:01, 716436.11it/s]

 39%|███▉      | 638

In [3]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

32768it [00:33, 969.48it/s]   
1654784it [00:32, 50574.07it/s]  
8192it [00:30, 265.42it/s]              


In [4]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [5]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
for epoch in range(1, 10):
    train(epoch)
    test()

: 0.116429
Train Epoch: 5 [47360/60000 (79%)]	Loss: 0.170364
Train Epoch: 5 [48000/60000 (80%)]	Loss: 0.250145
Train Epoch: 5 [48640/60000 (81%)]	Loss: 0.065797
Train Epoch: 5 [49280/60000 (82%)]	Loss: 0.103591
Train Epoch: 5 [49920/60000 (83%)]	Loss: 0.060383
Train Epoch: 5 [50560/60000 (84%)]	Loss: 0.031807
Train Epoch: 5 [51200/60000 (85%)]	Loss: 0.093381
Train Epoch: 5 [51840/60000 (86%)]	Loss: 0.106939
Train Epoch: 5 [52480/60000 (87%)]	Loss: 0.009591
Train Epoch: 5 [53120/60000 (88%)]	Loss: 0.059290
Train Epoch: 5 [53760/60000 (90%)]	Loss: 0.060678
Train Epoch: 5 [54400/60000 (91%)]	Loss: 0.076279
Train Epoch: 5 [55040/60000 (92%)]	Loss: 0.069673
Train Epoch: 5 [55680/60000 (93%)]	Loss: 0.039093
Train Epoch: 5 [56320/60000 (94%)]	Loss: 0.094104
Train Epoch: 5 [56960/60000 (95%)]	Loss: 0.064555
Train Epoch: 5 [57600/60000 (96%)]	Loss: 0.073879
Train Epoch: 5 [58240/60000 (97%)]	Loss: 0.019026
Train Epoch: 5 [58880/60000 (98%)]	Loss: 0.118428
Train Epoch: 5 [59520/60000 (99%)]	Loss